# Drunk Drivin' Fer Shuuure

In [1]:
import pandas as pd
import numpy as np
import requests as rq
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from collections import Counter
from matplotlib import style
style.use('ggplot')
sns.set(style="ticks")
from string import ascii_letters

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
sns.set(style="white")

Check out those nasty imports mm

In [2]:
df1, df2 = pd.read_csv('data/drunkdrivin_01.csv'), pd.read_csv('data/drunkdrivin_02.csv')
df3, df4 = pd.read_csv('data/drunkdrivin_03.csv'), pd.read_csv('data/drunkdrivin_04.csv')

FileNotFoundError: File b'data/drunkdrivin_01.csv' does not exist

We got data 4 days thanks to https://datausa.io/.

In [ ]:
joiner = pd.merge(df1, df2, how='left', on='geo_name')
joiner = pd.merge(joiner, df3, how='left', on='geo_name')
joiner = pd.merge(joiner, df4, how='left', on='geo_name')

Now to cherry pick the 2016 data:

In [ ]:
df = joiner[['geo_name', 'alcoholimpaired_driving_deaths_2016', 
             'income_below_poverty:pop_poverty_status_2016',
             'income_2016', 'income_below_poverty_2016', 'pop_poverty_status_2016',
             'unemployment_2016', 'uninsured_2016', 'high_school_graduation_2016',
             'mean_commute_minutes_2016', 'motor_vehicle_crash_deaths_2016',
             'population_living_in_a_rural_area_2016', 'severe_housing_problems_2016',
             'adult_obesity_2016', 'social_associations_2016', 'teen_births_2016',
             'violent_crime_2016', 'food_insecurity_2016', 'poor_mental_health_days_2016'
            ]].copy()

In [ ]:
df = df.dropna() # Who needs those null rows
len(df)

In [ ]:
prez = pd.read_csv('https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-16/master/2016_US_County_Level_Presidential_Results.csv')

In [ ]:
prez1 = prez.drop(['per_point_diff', 'diff'], axis=1)
prez1['trumpy'] = prez1['per_dem'] < prez1['per_gop']
prez1['trumpy'] = (prez1['trumpy'] == True).astype(int)

In [ ]:
counties = []
for i in df.values:
        counties.append(i[0].split(',')[0])

In [ ]:
df['geo_name'] = np.array(counties)

In [ ]:
prez1.columns = ['Unnamed: 0', 'votes_dem', 'votes_gop', 'total_votes', 'per_dem',
       'per_gop', 'state_abbr', 'geo_name', 'combined_fips', 'trumpy']

prez = prez1[['geo_name','trumpy']].copy()

In [ ]:
prez['geo_name'] = prez.geo_name.str.upper()
prez2 = prez[29:]

In [ ]:
df['geo_name'] = df.geo_name.str.upper()

In [ ]:
df = pd.merge(df, prez2.drop_duplicates(subset=['geo_name']), how='left', on='geo_name')

In [ ]:
features = df.iloc[:,1:]
target = df.iloc[:,0]

## Are our features normal?

In [ ]:
for column in features:
    features[column].plot.hist(normed=True, label = column+' histogram')
    features[column].plot.kde(label =column+' kde')
    plt.legend()
    plt.show()

Mostly, probably, we think.

### Are we feelin linear?

In [ ]:
# sns.pairplot(df) # This takes forever to run never run ever ever

### Is there multicolinearity?

In [ ]:
corr = features.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

### Nope not rly.